# Dataset clean

## Generic functions

In [1]:
def verbosity(msj, is_verbosity):
  if is_verbosity:
    print(msj)

def search_str_in_df(df_link, df_col, str_):
  return [(link, farm_size) for farm_size, link in zip(df_col, df_link) if type(farm_size)==str and str_ in farm_size]

## load file

In [2]:
import pandas as pd

file_path = '../data/AuctionsV3.xlsx'

try:
  df = pd.read_excel(file_path)
except FileNotFoundError:
  print(f"Error: File not found at {file_path}")
except Exception as e:
  print(f"An error occurred: {e}")




In [3]:
df.head(1)

,Unnamed: 0,Year,Country,Rank,Score,Farm,Url_farm,Farmer,Region,Weigth_(Kg),...,Aroma_Flavor,Acidity,Overall,Other,Farm_Size,Auction_Lot_Size,Auction_Lot_Size(Kg),Certifications,COMPANY NAME,City
0,0,2009,bolivia-2009,1,93.36,Agrotakesi SA,https://allianceforcoffeeexcellence.org/farm-d...,Mauricio Ramiro Diez de Medina,"Yanacahi, Yungas of La Paz",NaN,...,"perfume, jasmine, violet, honeysuckle, spices,...","brisk, citric, tartaric, intense, floral, tran...",NaN,"semi-sweet chocolate, vanilla, well-balanced, ...",2500,1818.81,NaN,NaN,NaN,Yanacahi


In [4]:
df.describe()

,Unnamed: 0,Year
count,5692.000000,5692.000000
mean,2845.500000,2014.466971
std,1643.283197,6.259609
min,0.000000,1999.000000
25%,1422.750000,2009.000000
50%,2845.500000,2015.000000
75%,4268.250000,2020.000000
max,5691.000000,2024.000000


In [5]:
# Order values counts by year
df['Year'].value_counts().sort_index()

Year
1999     10
2000     18
2001     33
2002     98
2003    111
2004    140
2005    206
2006    187
2007    181
2008    236
2009    228
2010    153
2011    282
2012    248
2013    212
2014    282
2015    339
2016    126
2017    397
2018    498
2019    257
2020    211
2021    357
2022    343
2023    333
2024    206
Name: count, dtype: int64

In [6]:
df['Country'].value_counts()

Country
brazil-pulped-naturals-2011    62
burundi-2018                   56
costa-rica-2018                47
brazil-naturals-2018           45
brazil-pulped-naturals-2003    43
                               ..
burundi-2013                   14
mexico-2024                    12
brazil-2023                    10
brazil-pulped-naturals-1999    10
mexico-2012                    10
Name: count, Length: 202, dtype: int64

In [7]:

df['Farm_Size'].value_counts().head(20)

Farm_Size
-     740
-1    357
10     82
8      39
4      36
14     32
3      32
5      31
70     28
20     28
9      27
6      27
2      26
12     24
15     24
7      23
,      23
30     22
50     22
21     19
Name: count, dtype: int64

In [8]:
# Remove "ha" from the 'Farm_Size' column
df['Farm_Size'] = df['Farm_Size'].str.replace('ha', '', case=False, regex=False)

In [9]:
# df[df['Farm_Size'].str.contains("hectares")]

In [10]:
df['Farm_Size'] = df['Farm_Size'].str.replace('hectares farm in production', '', case=False)

In [11]:
df['Farm_Size'] = df['Farm_Size'].str.replace('-?hec.*', '', case=False, regex=True)

In [12]:
df.loc[[4678, 4675]]

,Unnamed: 0,Year,Country,Rank,Score,Farm,Url_farm,Farmer,Region,Weigth_(Kg),...,Aroma_Flavor,Acidity,Overall,Other,Farm_Size,Auction_Lot_Size,Auction_Lot_Size(Kg),Certifications,COMPANY NAME,City
4678,4678,2021,nicaragua-2021,13,87.54,Los Milagros,https://farmdirectory.cupofexcellence.org/list...,Inversiones y desarrollo mozonte,Jinotega,169.65,...,"Floral , Black Tea , Bergamot, Chocolate, Citr...","Bergamot, Citric Acid, Dried Fruit, Lemon & Li...","Sweet Aromatics , Floral, Jucie like finished,...",NaN,74,NaN,NaN,NaN,NaN,NaN
4675,4675,2021,nicaragua-2021,10,88.04,Santa Ana,https://farmdirectory.cupofexcellence.org/list...,Aida Lila Zeledon Palacios,Jinotega,150,...,"Bittersweet Chocolate , Brown Sugar , Bakers C...","Berry , Citric Acid , Cherry, Citrus, Citrus F...","Good balance, Good body, Juicy, Sustained, Swe...",NaN,102,NaN,NaN,NaN,NaN,NaN


In [13]:
is_verbosity = False

# Function to clean and convert farm size
def clean_farm_size(link, size, index_):
  if size == "-":
    size = None

  if pd.isna(size):
      return size  # Return NaN values as they are

  size = str(size)

  if "m2" in size or "m²" in size or "metros cuadrados" in size:
    size = size.replace("m2", "").replace("m²", "").replace("metros cuadrados", "")
    aux_size = size
    size = size.replace(",", ".")
    verbosity(f"size m2 = {size}", is_verbosity)

    try:
      if float(size) <100:
        size = aux_size.replace(",", "")

      verbosity(f"Successfull changed size to ha = {float(size) / 10000}", is_verbosity)
      return float(size) / 10000
    except ValueError:
      verbosity(f"Error: {ValueError} in size: {size}", is_verbosity)
      return size # Return original string if conversion fails.
    
  if "patok" in size:
    size = size.replace("patok", "")
    aux_size = size
    size = size.replace(",", ".")
    verbosity(f"size patok = {size}", is_verbosity)

    try:
      verbosity(f"Successfull changed size to ha = {float(size) * 0.1}", is_verbosity)
      return float(size) * 0.1
    except ValueError:
      verbosity(f"Error: {ValueError} in size: {size}", is_verbosity)
      return size # Return original string if conversion fails.
    
  if "/ 2" in size:
    size = size.replace("/ 2", "")
    verbosity(f"size ha / ha_total = {size}", is_verbosity)

    try:
      verbosity(f"Successfull changed size to ha = {float(size)}", is_verbosity)
      return float(size)
    except ValueError:
      verbosity(f"Error: {ValueError} in size: {size}", is_verbosity)
      return size # Return original string if conversion fails.
  
  if "/163" in size:
    size = size.replace("/163", "")
    verbosity(f"size ha /... = {size}", is_verbosity)

    try:
      verbosity(f"Successfull changed size to ha = {float(size)}", is_verbosity)
      return float(size)
    except ValueError:
      verbosity(f"Error: {ValueError} in size: {size}", is_verbosity)
      return size # Return original string if conversion fails.

  if "MANZANAS" in size or "mzns" in size or "manzanas" in size or "mz" in size:
    size = size.replace("manzanas", "").replace("mzns", "").replace("MANZANAS", "").replace(",", "").replace("mz", "")
    verbosity(f"size mzn = {size}", is_verbosity)
    try:
      verbosity(f"Successfull changed size to ha = {float(size) * 0.698896}", is_verbosity)
      return float(size) * 0.698896
    except ValueError:
      if "y media" in size:
        size = size.replace("y media", "")
        try:
          verbosity(f"Successfull changed size to ha = {(float(size)+0.5) * 0.698896}", is_verbosity)
          return (float(size)+0.5) * 0.698896
        except ValueError as e:
          verbosity(f"Error: {e} in size: {size}", is_verbosity)
          return size # Return original string if conversion fails.
      verbosity(f"Error: {ValueError} in size: {size}", is_verbosity)
      return size # Return original string if conversion fails.

  try:
    float(size)
  except ValueError as e:
    if "," in size:
      try:
        size = size.replace(",", ".")
        return size
      except:
        print(f"Error: {e} in size: {size}")
        return size

    print(f"Second Error: {e} in size: {size}, index: {index_}, link {link}")
    return size

  return size

# [(link, value_) for value_, link in zip(df_col, df_link) if not(pd.isna(value_)) and type(float(value_))==float]
# Apply the function to the 'Farm_Size' column

df['Farm_size_he'] = pd.DataFrame( [clean_farm_size(link, value_, index_) for value_, link, index_ in zip(df['Farm_Size'], df['Url_farm'], df.index)])
# df['farm_size_he'] = df['Farm_Size'].apply(clean_farm_size, link=df['Url_farm'])




In [20]:
df.describe(include='all')

,Unnamed: 0,Year,Country,Rank,Score,Farm,Url_farm,Farmer,Region,Weigth_(Kg),...,Acidity,Overall,Other,Farm_Size,Auction_Lot_Size,Auction_Lot_Size(Kg),Certifications,COMPANY NAME,City,Farm_size_he
count,5692.000000,5692.000000,5692,5692,5604.0,5636,5100,5329,5377,1544,...,3668,1090,2519,3156,2747,82,120,58,3783,2416
unique,NaN,NaN,202,140,922.0,3675,4984,3910,1497,767,...,3327,969,2351,551,1406,59,20,40,1032,540
top,NaN,NaN,brazil-pulped-naturals-2011,10,0.0,La Esperanza,https://farmdirectory.cupofexcellence.org/list...,–,"Dipilto, Nueva Segovia",360,...,citric,"smooth, round, velvety, creamy, structured, co...",creamy mouthfeel,-,1521,Bensa,Organic,"Wataru & Co., Ltd.",Carmo de Minas,-1
freq,NaN,NaN,62,201,104.0,45,3,38,117,64,...,11,3,5,740,55,6,61,9,142,357
mean,2845.500000,2014.466971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1643.283197,6.259609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1999.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1422.750000,2009.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2845.500000,2015.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,4268.250000,2020.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df.head()

,Unnamed: 0,Year,Country,Rank,Score,Farm,Url_farm,Farmer,Region,Weigth_(Kg),...,Acidity,Overall,Other,Farm_Size,Auction_Lot_Size,Auction_Lot_Size(Kg),Certifications,COMPANY NAME,City,Farm_size_he
0,0,2009,bolivia-2009,1,93.36,Agrotakesi SA,https://allianceforcoffeeexcellence.org/farm-d...,Mauricio Ramiro Diez de Medina,"Yanacahi, Yungas of La Paz",NaN,...,"brisk, citric, tartaric, intense, floral, tran...",NaN,"semi-sweet chocolate, vanilla, well-balanced, ...",2500,1818.81,NaN,NaN,NaN,Yanacahi,2500
1,1,2009,bolivia-2009,2,92.05,Café Sima del Jaguar A,https://allianceforcoffeeexcellence.org/farm-d...,Braulio Luque Yana,"Caranavi, Yungas of La Paz",NaN,...,"lively, brilliant, currant-like, effervescent,...",NaN,"translucent, violet, rounded, complete, viscou...",13,2123.93,NaN,NaN,NaN,Caranavi,13
2,2,2009,bolivia-2009,3,91.62,Café Monterrey,https://allianceforcoffeeexcellence.org/farm-d...,Valentin Choquehuanca Aduviri,"Caranavi, Yungas of La Paz",NaN,...,"red currant-like, crisp, juicy, grape-like",NaN,"clarity, silky body, creamy mouthfeel, black c...",10,1740.33,NaN,NaN,NaN,Caranavi,10
3,3,2009,bolivia-2009,4,91.38,Café Jacaranda,https://allianceforcoffeeexcellence.org/farm-d...,Cruz Elias Choconapi,"Caranavi, Yungas of La Paz",NaN,...,"transparent, crisp, citric, grape-like, raspbe...",NaN,"balanced creamy body, buttery, syrupy, juicy, ...",10,1467.18,NaN,NaN,NaN,Caranavi,10
4,4,2009,bolivia-2009,5,90.9,Café Alan Coffe,https://allianceforcoffeeexcellence.org/farm-d...,Luis Yujra Arismende,"Caranavi, Yungas of La Paz",NaN,...,"citrus, crisp, sweet green grape, red berry-like",NaN,"complex, elegant, long aftertaste, full, plum",16,1591.74,NaN,NaN,NaN,Caranavi,16


In [25]:
def val_float_in_df(df_link, df_col):
  return [(link, value_) for value_, link in zip(df_col, df_link) if not(pd.isna(value_)) and type(float(value_))==float]
search_str_in_df(df['Url_farm'], df['Farm_size_he'], "m")
# val_float_in_df(df['Url_farm'], df['Farm_size_he'])

[]

In [26]:
df.to_excel('../data/AuctionsV4.xlsx')

In [ ]:
#Revisar caso row 4301
df.loc[[4289, 4298,4301]]